(analyzing-table-data)=
# Analyzing table data

Last time, we practiced loading, recoding, and tidying an example dataset. We also worked on a very limited example of exploratory analysis, constructing summary statistics and plots. Today, we'll push that analysis further, using Pandas to filter our data and ask whether some of our variables make a statistical difference to the outcomes we care about.

## From last time: loading and merging data
To begin follow the same approach as [last time](exploring-table-data) to download the data, load it, and combine it. This time, you _will not_ need to add the cumulative reward or cumulative trial variables. Instead, our goal will be to create a data frame that will allow us to model how total rewards earned vary across sexes as a function of behavioral state. These behavioral states are not observed but determined by using a [Hidden Markov Model](https://en.wikipedia.org/wiki/Hidden_Markov_model) (not as difficult as Wikipedia makes it look) to model behavior. That's a bit much for us to bite off at this point, but thanfully, the authors have already assigned each trial to either:
1. explore
2. exploit (left)
3. exploit (right)

```{admonition} Exercise
1. Use the split-apply-combine method to calculate total rewards earned for each animal for each session in each state.

2. Add `sex` back in as a column to the resulting data frame. There are multiple ways to do this, but a solution that works well in more complicated examples is to `merge` the resulting data frame from the previous question with a subset of columns from the original data and drop duplicate rows.
```

### Solution:

In [ ]:
%%bash
# recap what we did last time

curl --output tmp.zip -L -X GET "https://datadryad.org/api/v2/datasets/doi%3A10.5061%2Fdryad.z612jm6c0/download" -H "accept: application/zip"
unzip tmp.zip
unzip cleaned_up_restless_final_data.zip
rm tmp.zip
rm README.txt
rm cleaned_up_restless_final_data.zip
chmod -R u+wx cleaned\ up\ restless\ final\ data
mv cleaned\ up\ restless\ final\ data data

In [ ]:
import os
import pandas as pd

data_path = 'data/'

chunk_list = []
for (dirname, _, files) in list(os.walk(data_path)):
  folder = dirname.split('/')[-1]
  if 'session' in folder:
    session = int(folder[7:])  # make it an integer
    for fname in files:
      if fname.endswith('.csv'):
        sub = int(fname.split('.')[0])  # make it an integer
        chunk = pd.read_csv(dirname + '/' + fname, index_col=0)

        # add columns for session and subject
        chunk['session'] = session
        chunk['subject'] = sub
        if sub <= 16:
          chunk['sex'] = 'M'
        else:
          chunk['sex'] = 'F'
        
        # make a trial column by resetting the current index and renaming to trial
        chunk = chunk.reset_index()
        chunk = chunk.rename(columns={"index": "trial"})
        
        chunk_list.append(chunk)



# concatenate all data frames together

dat = pd.concat(chunk_list).reset_index(drop=True)  # overwrites our old dat variable!

In [ ]:
import shutil
shutil.rmtree('data')